In [1]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 7.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=e336e03728376626e92c1e323766ffa74e8b6ae5322618f9ecf10628280ee5aa
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=6d88c89ba3b46881450e738514816bef38a1a6259dcb3686f699f76eca0c9dd8
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [0]:
import numpy as np
from tensorflow import keras

In [0]:
import tensorflow as tf

In [4]:
tf.__version__

'2.2.0'

In [0]:
fashion = keras.datasets.fashion_mnist

In [6]:
(train_images, train_labels),(test_images, test_labels) = fashion.load_data()

4423680/4422102 [==============================] - 0s 0us/step


## Scaling the images

In [0]:
train_images = train_images/255.0
test_images = test_images/255.0

In [8]:
train_images.shape

(60000, 28, 28)

In [9]:
test_images.shape

(10000, 28, 28)

In [0]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

## Defining a function to create tuned model 

In [0]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [0]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [0]:
tuner_search = RandomSearch(build_model, 
                            objective='val_accuracy',
                            max_trials=5,
                            directory='output',
                            project_name='Mnist Fashion')

In [14]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.5005 - accuracy: 0.8295 - val_loss: 0.3943 - val_accuracy: 0.8525
Epoch 2/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3495 - accuracy: 0.8713 - val_loss: 0.3585 - val_accuracy: 0.8677
Epoch 3/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3104 - accuracy: 0.8848 - val_loss: 0.3474 - val_accuracy: 0.8778


Epoch 1/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.4332 - accuracy: 0.8442 - val_loss: 0.3332 - val_accuracy: 0.8740
Epoch 2/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.2868 - accuracy: 0.8950 - val_loss: 0.2727 - val_accuracy: 0.9010
Epoch 3/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.2377 - accuracy: 0.9116 - val_loss: 0.2737 - val_accuracy: 0.9038


Epoch 1/3
1688/1688 [==============================] - 12s 7ms/step - loss: 0.4957 - accuracy: 0.8270 - val_loss: 0.3615 - val_accuracy: 0.8610
Epoch 2/3
1688/1688 [==============================] - 12s 7ms/step - loss: 0.3592 - accuracy: 0.8661 - val_loss: 0.3595 - val_accuracy: 0.8698
Epoch 3/3
1688/1688 [==============================] - 11s 7ms/step - loss: 0.3274 - accuracy: 0.8775 - val_loss: 0.4328 - val_accuracy: 0.8387


Epoch 1/3
1688/1688 [==============================] - 11s 7ms/step - loss: 0.3978 - accuracy: 0.8560 - val_loss: 0.3047 - val_accuracy: 0.8850
Epoch 2/3
1688/1688 [==============================] - 11s 7ms/step - loss: 0.2626 - accuracy: 0.9030 - val_loss: 0.2590 - val_accuracy: 0.8995
Epoch 3/3
1688/1688 [==============================] - 11s 7ms/step - loss: 0.2090 - accuracy: 0.9224 - val_loss: 0.2546 - val_accuracy: 0.9072


Epoch 1/3
1688/1688 [==============================] - 11s 7ms/step - loss: 0.4857 - accuracy: 0.8336 - val_loss: 0.3578 - val_accuracy: 0.8638
Epoch 2/3
1688/1688 [==============================] - 11s 6ms/step - loss: 0.3492 - accuracy: 0.8725 - val_loss: 0.3663 - val_accuracy: 0.8692
Epoch 3/3
1688/1688 [==============================] - 11s 6ms/step - loss: 0.3172 - accuracy: 0.8827 - val_loss: 0.3840 - val_accuracy: 0.8572


INFO:tensorflow:Oracle triggered exit


In [18]:
tuner_search.get_best_models(num_models=1)[0].summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 80)        2080      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 48)        96048     
_________________________________________________________________
flatten (Flatten)            (None, 19200)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                1843296   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 1,942,394
Trainable params: 1,942,394
Non-trainable params: 0
_________________________________________________________________


In [19]:
model = tuner_search.get_best_models(num_models=1)[0]

In [20]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.1677 - accuracy: 0.9374 - val_loss: 0.2692 - val_accuracy: 0.9065
Epoch 5/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.1307 - accuracy: 0.9513 - val_loss: 0.2726 - val_accuracy: 0.9122
Epoch 6/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0977 - accuracy: 0.9633 - val_loss: 0.3030 - val_accuracy: 0.9103
Epoch 7/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0758 - accuracy: 0.9711 - val_loss: 0.3307 - val_accuracy: 0.9077
Epoch 8/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0615 - accuracy: 0.9775 - val_loss: 0.3601 - val_accuracy: 0.9082
Epoch 9/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0500 - accuracy: 0.9814 - val_loss: 0.4239 - val_accuracy: 0.9103
Epoch 10/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0385 - accuracy: 0.9859 - val_loss: 0.5041 - val_a